# ISYS613 - Data Sourcing and Quality
## Exam 1 - Fall 2022

## Instructions
For this exam, you are being asked to complete several unrelated
Python development activities. Each activity contains all of the information necessary to
complete the activity. If you believe that an unstated assumption is required to complete the
activity, please clarify your assumptions in the form of a comment.

For each of the questions below. Craft your solution in the single code cell that immediately
follows the question.

### Use only python standard library modules and packages (i.e., no 3'd party packages such as Pandas)
**You may use only python standard library modules** in the construction of your solutions.
If a python library module is not cataloged on The Python Standard Library web
page (https://docs.python.org/3.7/library/) then, you **may not use** the library module in
your solutions. I.e., no 3'd party packages such as Pandas.

### Solution Integrity
This exam is open-book, open-note. That said, you are to complete this exam
on your own.  Please do not discuss the exam with another individual.

In general, one should not use concepts not yet covered in our course to complete the activities below.
Similarly, do not use concepts you do not fully understand.  Translation - using a search engine
to construct your solutions is tedious, error prone, and most importantly, does not 
demonstrate your ability to solve problems!

## Question_1
Open and examine, but do not edit, the data in the BaseballPlayersData.csv file.  The file contains UTF-16 encoded
semi-colon (;) delimited data for over 1000 baseball players. Data for each player appears in the
rows of the file.  The following attributes are available for each player:

* name
* team
* position
* inchheight
* lbsweight,
* age
* 5 years of batting average data (batavg1, batavg2, batavg3, batavg4, batavg5).

### Programming Tasks

1. Using the csv python module, ingest the data from the BaseballPlayersData.csv file. Pay special
attention to the field data. Although the CSV informational standard indicates that space characters
appearing within a field are to be considered field data, in this case space characters should not be
considered as part of the field data.  Do not alter the input file to resolve this issue.
2. Which player by *name*, is the tallest (ie., greatest *inchheight*) and what is their height?
You may assume player names are unique.
3. Which player by *name* boasts the greatest 5 year batting average.  You may assume that each player
has played the same number of games in each of the 5 years of their batting average
data (ie, batavg1,..,batavg5) thus, making the "average of averages" a legitimate metric.
You may also assume all players have 5 years of batting average data and that the
values are interpretable as floating point numbers.

Author your solutions for each of the above tasks in the following code-cell.

In [52]:
import csv

# TEST DATA
in_file = './BaseballPlayersData.csv'


#2 Which player by name, is the tallest (ie., greatest inchheight) and what is their height? 
def tallest_player(data):
    max_height_player = max(data, key=lambda x: int(x['inchheight']))
    return max_height_player['name'], int(max_height_player['inchheight'])

#3 Which player by name boasts the greatest 5 year batting average.
def highest_avg_player(data):
    max_avg_player = max(data, key=lambda x: sum(float(x[f'batavg{i+1}']) for i in range(5)) / 5)
    return max_avg_player['name'], (sum(float(max_avg_player[f'batavg{i+1}']) for i in range(5)) / 5)


baseball_data = []

#1 
with open(in_file, mode='r', encoding='utf-16') as csv_file:
    csv_reader = csv.DictReader(csv_file, skipinitialspace=True,  delimiter=';')
    for row in csv_reader:
        # Remove leading and trailing spaces from each field
        cleaned_row = {key: value.strip() for key, value in row.items()}
        baseball_data.append(cleaned_row)


tallest_name, tallest_height = tallest_player(baseball_data)
print(f"The tallest player is {tallest_name} with a height of {tallest_height} inches.")

highest_avg_name, highest_avg = highest_avg_player(baseball_data)
print(f"The player with the highest 5-year batting average is {highest_avg_name} with an average of {highest_avg:.3f}.")

The tallest player is Jon_Rauch with a height of 83 inches.
The player with the highest 5-year batting average is Torii_Hunter with an average of 290.200.


## Question_2
Open and examine the data in the SpotifyAlbumTracks.XML file.  The file contains
the UTF-8 encoded XML results of a Spotify API query requesting the tracks(songs) from a **single album from
a single artist**.

### Programming Tasks

1. Using the *xml.etree.ElementTree* python standard library module, ingest the data from the
XML input file.
2. Author the code to find the *name* and *id* of the artist of the album. Note: these data
values appear multiple times in the XML data. It matters not which occurrence you use to
answer this question.
3. Author the code to find the name of the track having the longest
duration (ie, the largest *duration_ms* element)?

Author your solutions for each of the above tasks in the following code-cell.

In [53]:
import xml.etree.ElementTree as et


# TEST DATA
in_file = 'SpotifyAlbumTracks.XML'

# Question 1: Using the xml.etree.ElementTree python standard library module, ingest the data from the XML input file.
# Parse the XML file
tree = et.parse(in_file)
root = tree.getroot()

# Question 2: Author the code to find the name and id of the artist of the album.
artists_set = set()
for track in root.findall(".//track"):
    artist_element = track.find("artist")
    if artist_element is not None:
        artist_name = artist_element.find("name").text
        artist_id = artist_element.find("id").text
        artists_set.add((artist_name, artist_id))

# Question 3: Find the track with the longest duration
longest_track_element = max(root.findall(".//track"), key=lambda x: int(x.find("duration_ms").text))
track_name = longest_track_element.find("name").text
duration_ms = int(longest_track_element.find("duration_ms").text)


for artist_name, artist_id in artists_set:
    print(f"Artist: {artist_name}, ID: {artist_id}")

print(f"The longest track is '{track_name}' with a duration of {duration_ms} milliseconds.")

Artist: Elvis Presley, ID: 43ZHCT0cAZBISjO8DG9PnE
The longest track is 'That's When Your Heartaches Begin' with a duration of 201760 milliseconds.


## Question 3
The US Federal Government maintains many energy related Web-based resources.
One such API resource provides programmatic access to several renewable energy data categories.
One such resource is an API endpoint that supports access to alternative transportation technologies data.

In this question you will query the API designed to return information regarding alternative fuel
stations. This API returns information about alternative fuel station locations. Alternative fuel
 stations provide access to alternative fuels such as biodiesel, compressed natural gas,
 ethanol, electric charging, hydrogen, liquefied natural gas, and propane.

### Request Data Requirements
#### API Documentation
The alternative fuel stations API documentation can be seen at:

https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/

#### API URL
Use the following API url to access the alternative fuel station data:

```
https://developer.nrel.gov/api/alt-fuel-stations/v1
```

#### Required API Parameters
The following *alt-fuel-stations* GET request parameters **must** be provided as part of the request to access
the data and control the output file format:

Parameter Name | Required | Value | Description
----------|----------|-------|------------
format | Yes | Type: string | Request the *json* file format
api_key| Yes| Type: string | Use your private API key if you have one.  Otherwise, use the following key - *w2vQp5MnxJ2zrfJ9ZqaRcqZCnXOM7uPBx5WstoFL*

You may ignore all other *alt-fuel-stations* GET request parameters.

#### Response Data
Assume each row reflects the data for a unique alternative fuel station. You may also
assume all city names are unique (they are not; however, we will ignore that for now).

While the response data will contain many JSON elements, you may restrict your interest to the following:

Element | Description
------|------------
fuel_stations | array of alternative fuel station objects (FSO)
FSO.fuel_type_code | The type of alternative fuel the station provides. <u>Valid fuel type codes are shown below.</u>
FSO.station_name | The name of the station.
FSO.street_address | The street address of the station's location.
FSO.intersection_directions | Brief additional information about how to locate the station.
FSO.city | The city of the station's location.
FSO.state | The two-character code for the U.S. state or Canadian province/territory of the station's location.
FSO.zip | The ZIP code (postal code) of the station's location.

Fuel Type Codes | Description
----------------|------------
BD | Biodiesel
CNG | Compressed Natural Gas (CNG)
ELEC | Electric
E85 | Ethanol
HY | Hydrogen
LNG | Liquefied Natural Gas
LPG | Propane

### Programming Tasks
Author the python code necessary to answer the following questions:

1. Across all returned stations, what is the most commonly provided alternative fuel type? You may
ignore any ties.
2. Which zip code has the most alternative fuel stations that provide the ELEC fuel type? You may
ignore any ties.

In [54]:
import requests
import json

# TEST DATA
API_URL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1'
API_KEY = "w2vQp5MnxJ2zrfJ9ZqaRcqZCnXOM7uPBx5WstoFL"

# YOU MUST USE THE FOLLOWING DICTIONARY (parameters) AS THE params keyword argument to requests.get()
# Example:
# response_obj = requests.get(someurl, params=parameters, timeout=5)
# parameters = dict(format='json', api_key=API_KEY)
# Request parameters
params = {
    'format': 'json',
    'api_key': API_KEY
}

# Make the API request
response = requests.get(API_URL, params=params)
json_as_py = response.json()

#json_str = '{"station_locator_url":"https://afdc.energy.gov/stations/","total_results":17705,"station_counts":{"total":51167,"fuels":{"BD":{"total":0},"E85":{"total":372},"ELEC":{"total":50795,"stations":{"total":17333}},"HY":{"total":0},"LNG":{"total":0},"CNG":{"total":0},"LPG":{"total":0},"RD":{"total":0}}},"fuel_stations":[{"access_code":"private","access_days_time":"Fleet use only","access_detail_code":null,"cards_accepted":null,"date_last_confirmed":"2023-09-14","expected_date":null,"fuel_type_code":"ELEC","groups_with_access_code":"Private","id":1517,"open_date":"1999-10-15","owner_type_code":"LG","status_code":"E","restricted_access":null,"maximum_vehicle_class":null,"station_name":"LADWP - Truesdale Center","station_phone":null,"updated_at":"2023-09-14T14:01:49Z","facility_type":"UTILITY","geocode_status":"GPS","latitude":34.2483191527193,"longitude":-118.3879713743439,"city":"Sun Valley","intersection_directions":null,"plus4":null,"state":"CA","street_address":"11797 Truesdale St","zip":"91352","country":"US","bd_blends":null,"cng_dispenser_num":null,"cng_fill_type_code":null,"cng_psi":null,"cng_renewable_source":null,"cng_total_compression":null,"cng_total_storage":null,"cng_vehicle_class":null,"cng_has_rng":null,"e85_blender_pump":null,"e85_other_ethanol_blends":null,"ev_connector_types":["CHADEMO","J1772","J1772COMBO"],"ev_dc_fast_num":2,"ev_level1_evse_num":null,"ev_level2_evse_num":57,"ev_network":"SHELL_RECHARGE","ev_network_web":"https://shellrecharge.com/en-us/solutions","ev_other_evse":null,"ev_pricing":null,"ev_renewable_source":null,"ev_workplace_charging":true,"hy_is_retail":null,"hy_pressures":null,"hy_standards":null,"hy_status_link":null,"lng_renewable_source":null,"lng_vehicle_class":null,"lng_has_rng":null,"lpg_primary":null,"lpg_nozzle_types":null,"ng_fill_type_code":null,"ng_psi":null,"ng_vehicle_class":null,"rd_blends":null,"rd_blends_fr":null,"rd_blended_with_biodiesel":null,"rd_max_biodiesel_level":null,"nps_unit_name":null,"access_days_time_fr":null,"intersection_directions_fr":null,"bd_blends_fr":null,"groups_with_access_code_fr":"Privé","ev_pricing_fr":null},{"access_code":"private","access_days_time":null,"access_detail_code":null,"cards_accepted":null,"date_last_confirmed":"2023-01-10","expected_date":null,"fuel_type_code":"ELEC","groups_with_access_code":"Private","id":1519,"open_date":"2020-02-28","owner_type_code":"LG","status_code":"E","restricted_access":null,"maximum_vehicle_class":null,"station_name":"LADWP - West LA District Office","station_phone":null,"updated_at":"2023-02-15T22:45:41Z","facility_type":"UTILITY","geocode_status":"200-8","latitude":34.052542,"longitude":-118.448504,"city":"Los Angeles","intersection_directions":null,"plus4":null,"state":"CA","street_address":"1394 S Sepulveda Blvd","zip":"90024","country":"US","bd_blends":null,"cng_dispenser_num":null,"cng_fill_type_code":null,"cng_psi":null,"cng_renewable_source":null,"cng_total_compression":null,"cng_total_storage":null,"cng_vehicle_class":null,"cng_has_rng":null,"e85_blender_pump":null,"e85_other_ethanol_blends":null,"ev_connector_types":["J1772"],"ev_dc_fast_num":null,"ev_level1_evse_num":null,"ev_level2_evse_num":4,"ev_network":"Non-Networked","ev_network_web":null,"ev_other_evse":null,"ev_pricing":"Free","ev_renewable_source":null,"ev_workplace_charging":true,"hy_is_retail":null,"hy_pressures":null,"hy_standards":null,"hy_status_link":null,"lng_renewable_source":null,"lng_vehicle_class":null,"lng_has_rng":null,"lpg_primary":null,"lpg_nozzle_types":null,"ng_fill_type_code":null,"ng_psi":null,"ng_vehicle_class":null,"rd_blends":null,"rd_blends_fr":null,"rd_blended_with_biodiesel":null,"rd_max_biodiesel_level":null,"nps_unit_name":null,"access_days_time_fr":null,"intersection_directions_fr":null,"bd_blends_fr":null,"groups_with_access_code_fr":"Privé","ev_pricing_fr":null}]}'
#json_as_py = json.loads(json_str)
#print(json_as_py)

# Question 1: Across all returned stations, what is the most commonly provided alternative fuel type?
fuel_type_counts = {}
for station in data['fuel_stations']:
    fuel_type = station['fuel_type_code']
    fuel_type_counts[fuel_type] = fuel_type_counts.get(fuel_type, 0) + 1
print(fuel_type_counts)
most_common_fuel_type = max(fuel_type_counts, key=fuel_type_counts.get)

print(f"The most commonly provided alternative fuel type is: {most_common_fuel_type}")

# Question 2: Which zip code has the most alternative fuel stations that provide the ELEC fuel type?
elec_stations_by_zip = {}
for station in data['fuel_stations']:
    if station['fuel_type_code'] == 'ELEC':
        zip_code = station['zip']
        elec_stations_by_zip[zip_code] = elec_stations_by_zip.get(zip_code, 0) + 1

most_elec_zip = max(elec_stations_by_zip, key=elec_stations_by_zip.get)

print(f"The zip code with the most ELEC fuel stations is: {most_elec_zip}")

{'CNG': 1384, 'E85': 4489, 'ELEC': 67603, 'LNG': 121, 'LPG': 2839, 'BD': 1700, 'HY': 120, 'RD': 609}
The most commonly provided alternative fuel type is: ELEC
The zip code with the most ELEC fuel stations is: 94025
